In [ ]:
%%capture
%pip install yfinance
%pip install pysentiment2
%pip install transformers
%pip install py-readability-metrics
%python -m nltk.downloader punkt

In [1]:
import yfinance as yf
import pysentiment2 as ps
from bs4 import BeautifulSoup
import requests
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from datetime import datetime as dt
import spacy
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import MinMaxScaler
import torch
from readability import Readability
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [10]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import SGDClassifier
from sklearn import metrics

Load text to numpy:

In [19]:
from numpy import loadtxt
file = open('Feature_extracted_data.csv', 'rb')
data = np.loadtxt(file, delimiter=",",dtype='str')

df = pd.DataFrame(data)

mask = (df == 'nan').any(1)
df = df[~mask] #gets rid of all rows with string 'nan'
data = df.to_numpy()
print(data)

[['MMM' '2021' '4' ... '11.140535015026959' '34.04855319621302'
  '70.70256731749184']
 ['MMM' '2021' '3' ... '10.793553405168563' '38.32631380425198'
  '69.42418092957224']
 ['MMM' '2021' '1' ... '12.68835289967788' '44.57263896515957'
  '66.4627367720271']
 ...
 ['ZTS' '2015' '2' ... '15.112257680678326' '43.25211681196191'
  '45.4412662531803']
 ['ZTS' '2015' '1' ... '13.454273218885966' '40.4055555429274'
  '55.35886017280515']
 ['ZTS' '2014' '4' ... '12.68835289967788' '46.02614538683838'
  '58.878374642147904']]


In [3]:
#test print
data[0,:]

array(['MMM', '2021', '4', '-0.04189472305504671', '-0.14615007454974582',
       '-0.15708868256537967', '-0.13270961824249328',
       '-0.12258565616124262', '-0.046017860200147975',
       '-0.14982455628852887', '-0.16071609079135551',
       '-0.1364419400182546', '-0.12636146743188026',
       '0.8189584853752043', '95.0', '69.0', '0.15853658439916718',
       '0.08291203231399796', '101.0', '66.0', '0.2095808370683782',
       '0.0709732256391954', '14.317249209551537', '7.378095800032188',
       '18.112190935396296', '11.051034253014397', '16.404322709996244',
       '11.140535015026959', '34.04855319621302', '70.70256731749184'],
      dtype=object)

Dataset Split:

In [60]:
X_data = data[:,13:]
Y_data = data[:,5] 
#Y_data = data[:,5] looks at difference between Day 0 and Day 50
X_data = X_data.astype(np.float64)
Y_data = Y_data.astype(np.float64)

df = pd.DataFrame(Y_data)
#for classification, df.loc labels all values lower than or greater than to 0 or 1:
df.loc[(df[0] >= 0.10)] = 1
df.loc[(df[0] < 0.10)] = 0

# df.loc[((df[0] < 0.4) & (df[0] >= 0.03))] = 1
# df.loc[((df[0] < -0.03) & (df[0] >= -0.4))] = -1
# df.loc[((df[0] < 0.05) & (df[0] >= -0.05))] = 0

Y_data = df.to_numpy()


#scales the X datasets within a range of 0 to 1
scaler = MinMaxScaler()
X_data = scaler.fit_transform(X_data)


#splits the dataset into 67% training and 33% test
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.33) 

REGRESSION:

In [11]:
#reg1 stands for 1st regression model, these three regression models below suggests regression is not effective
#this is why classification models are used afterwards

reg1 = LinearRegression()
reg1.fit(X_train, Y_train)

Y_pred = reg1.predict(X_test)
ml1_score = reg1.score(X_test, Y_test)
ml1_r2_score = r2_score(Y_test, Y_pred)

print("MAE:",mean_absolute_error(Y_test, Y_pred))
print("MSE:",mean_squared_error(Y_test, Y_pred))
print("RMSE:",mean_squared_error(Y_test, Y_pred,squared=False))
print("R2:",r2_score(Y_test, Y_pred))
print("MAPE:",mean_absolute_percentage_error(Y_test, Y_pred))
df = pd.DataFrame({'Actual':Y_test.flatten(), 'Predicted': Y_pred.flatten()})

MAE: 0.0780341789002861
MSE: 0.013205152471813367
RMSE: 0.11491367399841225
R2: 0.0011494055008733906
MAPE: 77.34652149039927


In [75]:
reg2 = RandomForestRegressor(max_depth=2, random_state=0)
reg2.fit(X_train, Y_train)

Y_pred = reg2.predict(X_test)
score = reg2.score(X_test, Y_test)

print("MAE:",mean_absolute_error(Y_test, Y_pred))
print("MSE:",mean_squared_error(Y_test, Y_pred))
print("RMSE:",mean_squared_error(Y_test, Y_pred,squared=False))
print("R2:",r2_score(Y_test, Y_pred))
print("MAPE:",mean_absolute_percentage_error(Y_test, Y_pred))
df = pd.DataFrame({'Actual':Y_test.flatten(), 'Predicted': Y_pred.flatten()})

/var/folders/dr/g848f5b55mn9xp6p982hbg280000gn/T/ipykernel_35202/1902329584.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  reg2.fit(X_train, Y_train)


MAE: 0.07711656455004583
MSE: 0.012746905209592789
RMSE: 0.11290219311241384
R2: -0.00021209224811680194
MAPE: 123.01734024336159


In [74]:
reg3 = MLPRegressor(random_state=1, max_iter=1000).fit(X_train, Y_train)
reg3.fit(X_train, Y_train)

Y_pred = reg3.predict(X_test)
score = reg3.score(X_test, Y_test)

print("MAE:",mean_absolute_error(Y_test, Y_pred))
print("MSE:",mean_squared_error(Y_test, Y_pred))
print("RMSE:",mean_squared_error(Y_test, Y_pred,squared=False))
print("R2:",r2_score(Y_test, Y_pred))
print("MAPE:",mean_absolute_percentage_error(Y_test, Y_pred))
df = pd.DataFrame({'Actual':Y_test.flatten(), 'Predicted': Y_pred.flatten()})

/Users/victor/miniforge3/envs/NLP_env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1607: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/victor/miniforge3/envs/NLP_env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1607: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MAE: 0.07744414459404646
MSE: 0.012793316855438545
RMSE: 0.11310754552830923
R2: -0.0038538773428336004
MAPE: 96.18310129178005


CLASSIFICATION:

In [61]:
#Starting off with random forest classifer 
rfc = RandomForestClassifier(max_depth=1000, n_estimators=1000)

rfc.fit(X_train, Y_train)
score = rfc.score(X_test, Y_test)
Y_pred = rfc.predict(X_test)

print(metrics.classification_report(Y_test, Y_pred))

/var/folders/dr/g848f5b55mn9xp6p982hbg280000gn/T/ipykernel_43610/2539171930.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(X_train, Y_train)


              precision    recall  f1-score   support

         0.0       0.84      1.00      0.91      5047
         1.0       0.85      0.02      0.03       986

    accuracy                           0.84      6033
   macro avg       0.84      0.51      0.47      6033
weighted avg       0.84      0.84      0.77      6033



In [47]:
df = pd.DataFrame({'Actual':Y_test.flatten(), 'Predicted': Y_pred.flatten()})
df.head(30)

,Actual,Predicted
0,0.0,0.0
1,0.0,0.0
2,1.0,0.0
3,0.0,0.0
4,1.0,0.0
5,0.0,0.0
6,0.0,0.0
7,0.0,0.0
8,0.0,0.0
9,1.0,0.0


In [48]:
#this shows the probability of each categories
Y_pred = rfc.predict_proba(X_test)
df = pd.DataFrame({'probabilityA':Y_pred[:,0], 'probabilityB':Y_pred[:,1]})
df.head(30)

,probabilityA,probabilityB
0,0.873,0.127
1,0.770,0.230
2,0.752,0.248
3,0.841,0.159
4,0.825,0.175
5,0.834,0.166
6,0.791,0.209
7,0.883,0.117
8,0.765,0.235
9,0.859,0.141
